In [9]:
import pandas as pd
import numpy as np
#from sklearn import svm
from sklearn import ensemble
#from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import StratifiedKFold

C:\Users\Simon\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
def tpr_weight_funtion(y_true,y_predict):
    d = pd.DataFrame()
    d['prob'] = list(y_predict)
    d['y'] = list(y_true)
    d = d.sort_values(['prob'], ascending=[0])
    y = d.y
    PosAll = pd.Series(y).value_counts()[1]
    NegAll = pd.Series(y).value_counts()[0]
    pCumsum = d['y'].cumsum()
    nCumsum = np.arange(len(y)) - pCumsum + 1
    pCumsumPer = pCumsum / PosAll
    nCumsumPer = nCumsum / NegAll
    TR1 = pCumsumPer[abs(nCumsumPer-0.001).idxmin()]
    TR2 = pCumsumPer[abs(nCumsumPer-0.005).idxmin()]
    TR3 = pCumsumPer[abs(nCumsumPer-0.01).idxmin()]
    return 'TC_AUC',0.4 * TR1 + 0.3 * TR2 + 0.3 * TR3,True

"""=====================================================================================================================
1 读取数据
"""
path = "E:/data2/"
op_train = pd.read_csv(path+"operation_train_new.csv")
trans_train = pd.read_csv(path+"transaction_train_new.csv")

op_test = pd.read_csv(path+"operation_round1_new.csv")
trans_test = pd.read_csv(path+"transaction_round1_new.csv")
y = pd.read_csv(path+"tag_train_new.csv")

sub = pd.read_csv(path+"sub.csv")


def get_feature(op,trans,label): # 分别读取operation, transaction, y
    for feature in op.columns[2:]:
        label = label.merge(op.groupby(['UID'])[feature].count().reset_index(),on='UID',how='left')#先把y附加进dataframe, 对UID进行分组，分别统计各个UID出现的数量
        label =label.merge(op.groupby(['UID'])[feature].nunique().reset_index(),on='UID',how='left')#对UID进行分组，分别统计各个UID的不唯一的数量
    
    for feature in trans.columns[2:]:
        if trans_train[feature].dtype == 'object':# 判断是否是可以统计的属性
            label =label.merge(trans.groupby(['UID'])[feature].count().reset_index(),on='UID',how='left')
            label =label.merge(trans.groupby(['UID'])[feature].nunique().reset_index(),on='UID',how='left')
        else:
            label =label.merge(trans.groupby(['UID'])[feature].count().reset_index(),on='UID',how='left')
            label =label.merge(trans.groupby(['UID'])[feature].nunique().reset_index(),on='UID',how='left')
            label =label.merge(trans.groupby(['UID'])[feature].max().reset_index(),on='UID',how='left') #附加每个属性最大的
            label =label.merge(trans.groupby(['UID'])[feature].min().reset_index(),on='UID',how='left')#附加每个属性最小的
            label =label.merge(trans.groupby(['UID'])[feature].sum().reset_index(),on='UID',how='left')#附加求和
            label =label.merge(trans.groupby(['UID'])[feature].mean().reset_index(),on='UID',how='left')#附加均值
            label =label.merge(trans.groupby(['UID'])[feature].std().reset_index(),on='UID',how='left')#附加方差
    return label

C:\Users\Simon\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
train = get_feature(op_train,trans_train,y).fillna(-1)#用-1填充空格
test = get_feature(op_test,trans_test,sub).fillna(-1)

train = train.drop(['UID','Tag'],axis = 1).fillna(-1)
label = y['Tag']

test_id = test['UID']
test = test.drop(['UID','Tag'],axis = 1).fillna(-1)

skf = StratifiedKFold(n_splits=5, random_state=2018, shuffle=True)##5折价交差验证

oof_preds = np.zeros(train.shape[0])#做一个与训练集形状相同的矩阵
sub_preds = np.zeros(test_id.shape[0])#做一个与测试集Id形状相同的矩阵

    


0
1
2
3
4
0.7244340723453908


In [19]:
for index, (train_index, test_index) in enumerate(skf.split(train, label)):#用5折交叉验证切分train 和 label, 用enumerate给每一组数据加一个索引
    print(index)
    iTrees=50
    depth = None
    maxFeat  = 20 #try tweaking
    attackRFModel = ensemble.RandomForestClassifier(n_estimators=iTrees, max_depth=depth, max_features=maxFeat,
                                                 oob_score=False, random_state=531)
    attackRFModel.fit(train.iloc[train_index], label.iloc[train_index])
    oof_preds[test_index] = attackRFModel.predict_proba(train.iloc[test_index])[:,1]

    test_pred = attackRFModel.predict_proba(test)[:, 1]#取预测记过的第二列（预测为黑样本的概率）
    sub_preds += test_pred / 5 #5折交叉，每次取1/5
m = tpr_weight_funtion(y_predict=oof_preds,y_true=label)
print(m[1])
sub = pd.read_csv(path+"sub.csv")
sub['Tag'] = sub_preds
sub.to_csv(path+"rf_baseline_%s.csv"%str(m),index=False)

0
1
2
3
4
0.7219136522753792
